In [37]:
import heapq

class Event:
    '''
    Store the properties of one event in the Schedule class defined below. Each
    event has a time at which it needs to run, a function to call when running
    the event, along with the arguments and keyword arguments to pass to that
    function.
    '''
    def __init__(self, timestamp, function, *args, **kwargs):
        self.timestamp = timestamp
        self.function = function
        self.args = args
        self.kwargs = kwargs

    def __lt__(self, other):
        '''
        This overloads the less-than operator in Python. We need it so the
        priority queue knows how to compare two events. We want events with
        earlier (smaller) times to go first.
        '''
        return self.timestamp < other.timestamp

    def run(self, schedule):
        '''
        Run an event by calling the function with its arguments and keyword
        arguments. The first argument to any event function is always the
        schedule in which events are being tracked. The schedule object can be
        used to add new events to the priority queue.
        '''
        self.function(schedule, *self.args, **self.kwargs)


class Schedule:
    '''
    Implement an event schedule using a priority queue. You can add events and
    run the next event.
    
    The `now` attribute contains the time at which the last event was run.
    '''
    
    def __init__(self):
        self.now = 0  # Keep track of the current simulation time
        self.priority_queue = []  # The priority queue of events to run
    
    def add_event_at(self, timestamp, function, *args, **kwargs):
        # Add an event to the schedule at a particular point in time.
        heapq.heappush(
            self.priority_queue,
            Event(timestamp, function, *args, **kwargs))
    
    def add_event_after(self, interval, function, *args, **kwargs):
        # Add an event to the schedule after a specified time interval.
        self.add_event_at(self.now + interval, function, *args, **kwargs)
    
    def next_event_time(self):
        # Return the time of the next event. The `now` attribute of this class
        # contain the time of the last event that was run.
        return self.priority_queue[0].timestamp

    def run_next_event(self):
        # Get the next event from the priority queue and run it.
        event = heapq.heappop(self.priority_queue)
        self.now = event.timestamp
        event.run(self)
        
    def __repr__(self):
        return (
            f'Schedule() at time {self.now} ' +
            f'with {len(self.priority_queue)} events in the queue')
    
    def print_events(self):
        # Print out diagnostic information about the events in the schedule.
        print(repr(self))
        for event in sorted(self.priority_queue):
            print(f'   {event.timestamp}: {event.function.__name__}')
            

In [38]:
from scipy.stats import expon

class Queue:
    
    def __init__(self, size, service_rate, served = False):
        
        #size of the queue
        self.size = size
        #keeps track if the line is currently serving
        self.served = served
        #service rate of the queue
        self.service_rate = service_rate
        
    def start_serving(self, schedule):
        
        #check if there is someone is line and if the server is free
        if self.served == False and self.size > 0:
            #schedule end of serving
            schedule.add_event_after(1/self.service_rate, self.end_serving)
            #update queue size and set served to true 
            self.size -= 1
            self.served = True
            
            print(f'{schedule.now}: new costumer being served. New queue length is {self.size}')
            
    def end_serving(self, schedule):
        
        #end serving by setting served to false 
        self.served = False
        print(f'{schedule.now}: finished serving')
        
        #start serving again if queue is not empty
        if self.size > 0:
            self.start_serving(schedule)
        
            

class BusSystem:
    
    def __init__(self, arrival_rate, queue):
        
        #store arrival rate and queue
        self.arrival_rate = arrival_rate
        self.queue = queue
        
    def start(self, schedule):
        #schedule initial arriveing
        schedule.add_event_after(expon.rvs(1/self.arrival_rate), self.arrival)
        
    def arrival(self, schedule):
        #schedule next arrival
        schedule.add_event_after(expon.rvs(1/self.arrival_rate), self.arrival)
        #update queue size
        self.queue.size += 1
        
        print(f'{schedule.now}: costumer arriving. New queue length is {self.queue.size}')
        
        #if the server is free start serving
        if self.queue.served == False:
            self.queue.start_serving(schedule)
        
    
    
    
def run_simulation(arrival_rate, service_rate, run_until):
    
    #create schedule queue and bus system
    q = Queue(0, service_rate)
    bussystem = BusSystem( arrival_rate, q)
    schedule = Schedule()
    bussystem.start(schedule)
    
    #run simmulation is set time
    while schedule.next_event_time() <= run_until:
        schedule.run_next_event()

In [39]:
#example simulation
run_simulation(5, 0.5, 20)

0.48950318828022904: costumer arriving. New queue length is 1
0.48950318828022904: new costumer being served. New queue length is 0
1.4725149247625688: costumer arriving. New queue length is 1
1.8640285373020875: costumer arriving. New queue length is 2
2.489503188280229: finished serving
2.489503188280229: new costumer being served. New queue length is 1
2.5355764726530228: costumer arriving. New queue length is 2
2.8193897586451406: costumer arriving. New queue length is 3
3.8080082317963457: costumer arriving. New queue length is 4
4.489141512617975: costumer arriving. New queue length is 5
4.48950318828023: finished serving
4.48950318828023: new costumer being served. New queue length is 4
6.48950318828023: finished serving
6.48950318828023: new costumer being served. New queue length is 3
7.896544092058888: costumer arriving. New queue length is 4
8.21938362762252: costumer arriving. New queue length is 5
8.48950318828023: finished serving
8.48950318828023: new costumer being serv